<a href="https://colab.research.google.com/github/gimquokka/DACON_Jeju_credit_card_bigdata_competition/blob/master/KJY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 제주 빅데이터 경진대회 - 김진영

## 1. Import Library



In [2]:
!pip3 install catboost
!pip3 install category_encoders

     |████████████████████████████████| 64.8MB 61kB/s 
     |████████████████████████████████| 81kB 2.1MB/s 


In [3]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from catboost import CatBoostRegressor, Pool
import category_encoders as ce

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
print('Pandas : %s'%(pd.__version__))
print('Numpy : %s'%(np.__version__))
print("Scikit-Learn : %s"%(sklearn.__version__))
!python --version

Pandas : 1.0.5
Numpy : 1.18.5
Scikit-Learn : 0.22.2.post1
Python 3.6.9


## 2. Data Cleansing & Pre-**Processing**🧹

### 2.1 Mount training dataset and it's info

---



In [5]:
# Mount dataset from gooogle drive
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [6]:
!ls /gdrive/My\ Drive/data/dacon_Jeju_credit_card/

201901-202003.csv  submission.csv


In [7]:
# Check data
data = pd.read_csv('/gdrive/My Drive/data/dacon_Jeju_credit_card/201901-202003.csv')

In [8]:
# Training data information
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24697792 entries, 0 to 24697791
Data columns (total 12 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   REG_YYMM      int64 
 1   CARD_SIDO_NM  object
 2   CARD_CCG_NM   object
 3   STD_CLSS_NM   object
 4   HOM_SIDO_NM   object
 5   HOM_CCG_NM    object
 6   AGE           object
 7   SEX_CTGO_CD   int64 
 8   FLC           int64 
 9   CSTMR_CNT     int64 
 10  AMT           int64 
 11  CNT           int64 
dtypes: int64(6), object(6)
memory usage: 2.2+ GB


In [9]:
# See few of top listed data from dataset
display(data.head())
# See few of below listed data from dataset
display(data.tail())
display(data[100:105])

REG_YYMM CARD_SIDO_NM CARD_CCG_NM STD_CLSS_NM  ... FLC CSTMR_CNT      AMT  CNT
0    201901           강원         강릉시  건강보조식품 소매업  ...   1         4   311200    4
1    201901           강원         강릉시  건강보조식품 소매업  ...   2         7  1374500    8
2    201901           강원         강릉시  건강보조식품 소매업  ...   2         6   818700    6
3    201901           강원         강릉시  건강보조식품 소매업  ...   3         4  1717000    5
4    201901           강원         강릉시  건강보조식품 소매업  ...   4         3  1047300    3

[5 rows x 12 columns]

REG_YYMM CARD_SIDO_NM CARD_CCG_NM  ... CSTMR_CNT     AMT CNT
24697787    202003           충북         충주시  ...         3   43300   4
24697788    202003           충북         충주시  ...         3   35000   3
24697789    202003           충북         충주시  ...         4  188000   6
24697790    202003           충북         충주시  ...         4   99000   6
24697791    202003           충북         충주시  ...         3  194000   3

[5 rows x 12 columns]

REG_YYMM CARD_SIDO_NM CARD_CCG_NM  ... CSTMR_CNT     AMT CNT
100    201901           강원         강릉시  ...         5  471000   5
101    201901           강원         강릉시  ...         3  153000   4
102    201901           강원         강릉시  ...         3   81000   3
103    201901           강원         강릉시  ...         5  374000   5
104    201901           강원         강릉시  ...         3  104000   3

[5 rows x 12 columns]

### 2.2 Separate REG_YYMM column into YYYY and MM column

In [10]:
def grap_year(data):
  '''
  년월 컬럼에서 년도를 추출하는 함수
  
  ex) 
  >>> data = [201901, 강원, 강릉시, ...]
  >>> grap_year(data[0])
  >>> 2019
  '''
  data = str(data)
  return int(data[:4])


def grap_month(data):
  '''
  년월 컬럼에서 월을 추출하는 함수

  ex) 
  >>> data = [201901, 강원, 강릉시, ...]
  >>> grap_year(data[0])
  >>> 01
  '''
  data = str(data)
  return int(data[4:])

In [11]:
data['year'] = data['REG_YYMM'].apply(lambda x: grap_year(x))
data['month'] = data['REG_YYMM'].apply(lambda x: grap_month(x))
data = data.drop(['REG_YYMM'], axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24697792 entries, 0 to 24697791
Data columns (total 13 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   CARD_SIDO_NM  object
 1   CARD_CCG_NM   object
 2   STD_CLSS_NM   object
 3   HOM_SIDO_NM   object
 4   HOM_CCG_NM    object
 5   AGE           object
 6   SEX_CTGO_CD   int64 
 7   FLC           int64 
 8   CSTMR_CNT     int64 
 9   AMT           int64 
 10  CNT           int64 
 11  year          int64 
 12  month         int64 
dtypes: int64(7), object(6)
memory usage: 2.4+ GB


### 2.3 Process missing value(NaN)

In [12]:
# Total num of missing value in training data
data.isnull().sum()

CARD_SIDO_NM         0
CARD_CCG_NM      87213
STD_CLSS_NM          0
HOM_SIDO_NM          0
HOM_CCG_NM      147787
AGE                  0
SEX_CTGO_CD          0
FLC                  0
CSTMR_CNT            0
AMT                  0
CNT                  0
year                 0
month                0
dtype: int64

In [13]:
# Pluck out row containing NaN.
# To see acturly how they looks like!
is_NaN = data.isnull()
row_has_NaN = is_NaN.any(axis=1)
row_with_NaN = data[row_has_NaN]

print(row_with_NaN.shape) # Total number of row that containing missing value

(227264, 13)


In [14]:
## Example of row containing NaN
display(row_with_NaN[:5])
display(row_with_NaN[-5:])

CARD_SIDO_NM CARD_CCG_NM   STD_CLSS_NM  ... CNT  year month
420            강원         강릉시   과실 및 채소 소매업  ...   3  2019     1
421            강원         강릉시   과실 및 채소 소매업  ...   4  2019     1
1189           강원         강릉시  기타 대형 종합 소매업  ...   3  2019     1
1190           강원         강릉시  기타 대형 종합 소매업  ...   6  2019     1
1191           강원         강릉시  기타 대형 종합 소매업  ...   5  2019     1

[5 rows x 13 columns]

CARD_SIDO_NM CARD_CCG_NM STD_CLSS_NM  ... CNT  year month
24697496           충북         충주시     한식 음식점업  ...   8  2020     3
24697497           충북         충주시     한식 음식점업  ...  28  2020     3
24697498           충북         충주시     한식 음식점업  ...  35  2020     3
24697499           충북         충주시     한식 음식점업  ...   4  2020     3
24697500           충북         충주시     한식 음식점업  ...   6  2020     3

[5 rows x 13 columns]

### 2.4 Data cleaning

In [15]:
df = data.copy()
df = df.drop(['CARD_CCG_NM', 'HOM_CCG_NM'], axis=1)

columns = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'year', 'month']
df = df.groupby(columns).sum().reset_index(drop=False)
df.info()
display(df[:10])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1057394 entries, 0 to 1057393
Data columns (total 11 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   CARD_SIDO_NM  1057394 non-null  object
 1   STD_CLSS_NM   1057394 non-null  object
 2   HOM_SIDO_NM   1057394 non-null  object
 3   AGE           1057394 non-null  object
 4   SEX_CTGO_CD   1057394 non-null  int64 
 5   FLC           1057394 non-null  int64 
 6   year          1057394 non-null  int64 
 7   month         1057394 non-null  int64 
 8   CSTMR_CNT     1057394 non-null  int64 
 9   AMT           1057394 non-null  int64 
 10  CNT           1057394 non-null  int64 
dtypes: int64(7), object(4)
memory usage: 88.7+ MB


CARD_SIDO_NM STD_CLSS_NM HOM_SIDO_NM  AGE  ...  month  CSTMR_CNT     AMT  CNT
0           강원  건강보조식품 소매업          강원  20s  ...      1          4  311200    4
1           강원  건강보조식품 소매업          강원  20s  ...      2          3  605000    3
2           강원  건강보조식품 소매업          강원  20s  ...      6          3  139000    3
3           강원  건강보조식품 소매업          강원  20s  ...      8          3   27500    3
4           강원  건강보조식품 소매업          강원  20s  ...      9          3  395500    3
5           강원  건강보조식품 소매업          강원  20s  ...     11         13  843100   16
6           강원  건강보조식품 소매업          강원  20s  ...      1          6  168500    6
7           강원  건강보조식품 소매업          강원  20s  ...      3          3  427510    2
8           강원  건강보조식품 소매업          강원  20s  ...      2          4  912000    5
9           강원  건강보조식품 소매업          강원  20s  ...      3         12  631000   13

[10 rows x 11 columns]

In [16]:
df.dtypes

CARD_SIDO_NM    object
STD_CLSS_NM     object
HOM_SIDO_NM     object
AGE             object
SEX_CTGO_CD      int64
FLC              int64
year             int64
month            int64
CSTMR_CNT        int64
AMT              int64
CNT              int64
dtype: object

In [17]:
# Label encoding

dtypes = df.dtypes
label_encoders = {}
for column in df.columns:
  if str(dtypes[column]) == 'object':
    encoder = LabelEncoder()
    encoder.fit(df[column])
    label_encoders[column] = encoder
  

df_label = df.copy()
for column in label_encoders.keys():
  encoder = label_encoders[column]
  df_label[column] = encoder.transform(df[column])

In [18]:
df_label.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1057394 entries, 0 to 1057393
Data columns (total 11 columns):
 #   Column        Non-Null Count    Dtype
---  ------        --------------    -----
 0   CARD_SIDO_NM  1057394 non-null  int64
 1   STD_CLSS_NM   1057394 non-null  int64
 2   HOM_SIDO_NM   1057394 non-null  int64
 3   AGE           1057394 non-null  int64
 4   SEX_CTGO_CD   1057394 non-null  int64
 5   FLC           1057394 non-null  int64
 6   year          1057394 non-null  int64
 7   month         1057394 non-null  int64
 8   CSTMR_CNT     1057394 non-null  int64
 9   AMT           1057394 non-null  int64
 10  CNT           1057394 non-null  int64
dtypes: int64(11)
memory usage: 88.7 MB


In [19]:
# CARD_SIDO_NM One Hot Encoding

onehot_encoder = OneHotEncoder(handle_unknown='ignore')
enc_df = pd.DataFrame(onehot_encoder.fit_transform(df_label[['CARD_SIDO_NM']]).toarray().astype(np.int))
CARD_SIDO_NM= data['CARD_SIDO_NM'].unique()
df_onehot = df_label.copy()
for i in range(len(enc_df.columns)):
  df_onehot.insert(i, CARD_SIDO_NM[i],enc_df.iloc[:,i:i+1])
df_onehot.head()

강원  경기  경남  경북  광주  대구  ...  FLC  year  month  CSTMR_CNT     AMT  CNT
0   1   0   0   0   0   0  ...    1  2019      1          4  311200    4
1   1   0   0   0   0   0  ...    1  2019      2          3  605000    3
2   1   0   0   0   0   0  ...    1  2019      6          3  139000    3
3   1   0   0   0   0   0  ...    1  2019      8          3   27500    3
4   1   0   0   0   0   0  ...    1  2019      9          3  395500    3

[5 rows x 28 columns]

In [21]:
#Binary encoding
df_binary = df_label.copy()

encoder = ce.BinaryEncoder(cols=['CARD_SIDO_NM'])

df_binary = encoder.fit_transform(df_binary)

df_binary.head()

CARD_SIDO_NM_0  CARD_SIDO_NM_1  CARD_SIDO_NM_2  ...  CSTMR_CNT     AMT  CNT
0               0               0               0  ...          4  311200    4
1               0               0               0  ...          3  605000    3
2               0               0               0  ...          3  139000    3
3               0               0               0  ...          3   27500    3
4               0               0               0  ...          3  395500    3

[5 rows x 16 columns]

In [22]:
# Explore encoded feature
# Label encoding
for key in label_encoders.keys():
  print(label_encoders[key].classes_)
  print(len(label_encoders[key].classes_))
  print(label_encoders[key].transform(label_encoders[key].classes_), '\n\n')

# One-hot encoding
print(onehot_encoder.categories_)
print(enc_df)

['강원' '경기' '경남' '경북' '광주' '대구' '대전' '부산' '서울' '세종' '울산' '인천' '전남' '전북'
 '제주' '충남' '충북']
17
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16] 


['건강보조식품 소매업' '골프장 운영업' '과실 및 채소 소매업' '관광 민예품 및 선물용품 소매업'
 '그외 기타 분류안된 오락관련 서비스업' '그외 기타 스포츠시설 운영업' '그외 기타 종합 소매업' '기타 대형 종합 소매업'
 '기타 수상오락 서비스업' '기타 외국식 음식점업' '기타 주점업' '기타음식료품위주종합소매업' '내항 여객 운송업' '마사지업'
 '면세점' '버스 운송업' '비알콜 음료점업' '빵 및 과자류 소매업' '서양식 음식점업' '수산물 소매업' '슈퍼마켓'
 '스포츠 및 레크레이션 용품 임대업' '여관업' '여행사업' '욕탕업' '육류 소매업' '일반유흥 주점업' '일식 음식점업'
 '자동차 임대업' '전시 및 행사 대행업' '정기 항공 운송업' '중식 음식점업' '차량용 가스 충전업' '차량용 주유소 운영업'
 '체인화 편의점' '택시 운송업' '피자 햄버거 샌드위치 및 유사 음식점업' '한식 음식점업' '호텔업'
 '화장품 및 방향제 소매업' '휴양콘도 운영업']
41
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40] 


['강원' '경기' '경남' '경북' '광주' '대구' '대전' '부산' '서울' '세종' '울산' '인천' '전남' '전북'
 '제주' '충남' '충북']
17
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16] 


['10s' '20s' '30s' '40s' '50s' '60s' '70s']
7
[0 1 2 3 4 5 6] 


[a

## 3. Exploratory Data Analysis

In [ ]:
# 입력하세요


## 4. Feature Engineering & Initial Modeling



In [40]:
# Setting features, target for training model

is_202003 = (df_label['month'] == 3) & (df_label['year'] == 2020)

trainset = df_label[~is_202003].sample(frac=1, random_state=0)
testset = df_label[is_202003].sample(frac=1, random_state=0)

In [ ]:
# Result of sample() to train_num
train_num.head()

In [ ]:
from sklearn.model_selection import train_test_split , GridSearchCV

# Remove 'CSTMR_CNT', 'AMT', 'CNT' column from train_feature
# train_features = train_num.drop(['CSTMR_CNT', 'AMT', 'CNT'], axis = 1)
train_features = train_num.drop(['CARD_SIDO_NM','AMT'], axis = 1)
# Take a ln(x+1) to AMT because, It's range is too large compare with the other value
# Ex) ln(5000) = 8.5...
train_target = np.log1p(train_num['AMT'])

2020년 3월 데이터를 예측데이터로 설정

In [ ]:
df_sample = train_num.drop(['CARD_SIDO_NM'], axis = 1).copy()
test = df_sample[(df_sample['year'] == 2020) & (df_sample['month'] == 3)]
test.info()

## 5. Model Tuning & Evaluation

In [ ]:
pool=Pool(data=train_features,label=train_target)
print (train_features.shape)

In [ ]:
model = CatBoostRegressor()

In [ ]:
test_features = test.drop(['AMT',], axis=1)
test_target = np.log1p(test['AMT'])
eval_set=(test_features,test_target)
# model.fit(train_features,train_target,use_best_model=True,eval_set=eval_set)

In [ ]:
# 훈련
model = RandomForestRegressor(n_estimators=10, n_jobs=10, random_state=0)
model.fit(train_features, train_target)

## 6. Conclusion & Discussion

In [ ]:
# 예측 템플릿 만들기
CARD_SIDO_NMs = df['CARD_SIDO_NM'].unique()
STD_CLSS_NMs  = df['STD_CLSS_NM'].unique()
HOM_SIDO_NMs  = df['HOM_SIDO_NM'].unique()
AGEs          = df['AGE'].unique()
SEX_CTGO_CDs  = df['SEX_CTGO_CD'].unique()
FLCs          = df['FLC'].unique()
years         = [2020]
months        = [4, 7]

temp = []
for CARD_SIDO_NM in CARD_SIDO_NMs:
    for STD_CLSS_NM in STD_CLSS_NMs:
        for HOM_SIDO_NM in HOM_SIDO_NMs:
            for AGE in AGEs:
                for SEX_CTGO_CD in SEX_CTGO_CDs:
                    for FLC in FLCs:
                        for year in years:
                            for month in months:
                                temp.append([CARD_SIDO_NM, STD_CLSS_NM, HOM_SIDO_NM, AGE, SEX_CTGO_CD, FLC, year, month])
temp = np.array(temp)
temp = pd.DataFrame(data=temp, columns=train_features.columns)

In [ ]:
# 예측
pred = model.predict(temp)
pred = np.expm1(pred)
temp['AMT'] = np.round(pred, 0)
temp['REG_YYMM'] = temp['year']*100 + temp['month']
temp = temp[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
temp = temp.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)

In [ ]:
# 디코딩 
temp['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(temp['CARD_SIDO_NM'])
temp['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(temp['STD_CLSS_NM'])

In [ ]:
# 제출 파일 만들기
submission = pd.read_csv('data/submission.csv', index_col=0)
submission = submission.drop(['AMT'], axis=1)
submission = submission.merge(temp, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission.index.name = 'id'
submission.to_csv('submission.csv', encoding='utf-8-sig')
submission.head()